In [1]:
# === REPORT GENERATOR ===
# 07_report.ipynb
#
# Funktion:
# Dieses Notebook ist der Abschluss der Pipeline. Es aggregiert Ergebnisse aus allen
# vorherigen Schritten (Training, Evaluation, Costed Backtest) und erstellt:
# 1. Einen menschenlesbaren Markdown-Bericht (REPORT_block7.md).
# 2. Ein JSON-File mit allen KPIs für maschinelle Weiterverarbeitung.
# 3. Einen Vergleich mit einfachen Benchmarks (LogReg, MACD), um die Leistung einzuordnen.

import os, json, yaml, re
from pathlib import Path
from datetime import datetime
import numpy as np
import pandas as pd

In [2]:
# === HELFER: RUN FINDEN ===
# Wir müssen wissen, welches Experiment wir auswerten sollen.
# Dazu suchen wir den neuesten Ordner, der zu unserer aktuellen Konfiguration passt.

def jread(p: Path):
    # Hilfsfunktion zum sicheren Laden von JSON
    with open(p, "r", encoding="utf-8") as f:
        return json.load(f)

def latest_lstm_run(results_dir: Path,
                    lookback: int = None,
                    horizon: int = None,
                    eps_mode: str = None,
                    epsilon: float = None,
                    strict: bool = False) -> Path | None:
    # Liste aller LSTM-Runs holen, neueste zuerst
    runs = sorted(results_dir.glob("*_lstm"), key=lambda p: p.stat().st_mtime, reverse=True)
    if not runs:
        return None

    # Filter-Logik: Wir prüfen, ob die Config des Runs zu unseren Anforderungen passt
    def matches(run: Path) -> bool:
        try:
            cfg = jread(run / "config.json")
        except Exception:
            return False # Defekte Config ignorieren
            
        # Parameter vergleichen (falls gefordert)
        ok_lb = (lookback is None) or (int(cfg.get("lookback", -1)) == lookback)
        
        # Dateinamen parsen, falls Parameter nicht direkt in Config stehen
        tc = str(cfg.get("train_csv", ""))
        mH = re.search(r"_cls_h(\d+)_", tc)
        mE = re.search(r"_(abs|rel)([\dp]+)\.csv$", tc)
        
        ok_h = True if horizon is None else (int(cfg.get("horizon", -1)) == horizon or (mH and int(mH.group(1)) == horizon))
        ok_m = True if eps_mode is None else ((mE and mE.group(1) == eps_mode))
        
        # Epsilon Check (komplizierter wegen Fließkomma-Vergleich)
        ok_e = True
        if epsilon is not None:
            if mE:
                ok_e = float(mE.group(2).replace("p",".")) == float(epsilon)
            else:
                ok_e = float(cfg.get("epsilon", 1e9)) == float(epsilon)
                
        return ok_lb and ok_h and ok_m and ok_e

    # Suchen
    matches_list = [r for r in runs if matches(r)]
    if matches_list:
        return matches_list[0]
    
    # Wenn strict=False, geben wir zur Not einfach den allerneuesten Run zurück
    return None if strict else runs[0]

In [3]:
# === CONFIG LADEN ===
ROOT = Path("..").resolve()
with open(ROOT / "config.json", "r") as f:
    C = json.load(f)

RESULTS_DIR = Path(C.get("results_dir", "../results")).resolve()
LOOKBACK    = int(C["lookback"])
FEATURESET  = C.get("featureset", "v2")

# Wir laden auch die Feature-Definitionen, um Label-Infos zu bekommen
HORIZON = MODE = EPS = None
yml = ROOT / f"data/features_{FEATURESET}.yml"
if yml.exists():
    meta = yaml.safe_load(open(yml, "r")) or {}
    lab = meta.get("label", {})
    HORIZON = int(lab.get("horizon", 0)) or None
    MODE    = str(lab.get("mode", "")) or None
    EPS     = float(lab.get("epsilon", 0.0)) or None

In [4]:
# === RUN-VERZEICHNIS BESTIMMEN ===
# Man kann RUN_DIR auch per Environment-Variable erzwingen (für Pipeline-Automation)
run_override = os.getenv("RUN_DIR", "").strip() or None

if run_override:
    RUN_DIR = Path(run_override).resolve()
else:
    RUN_DIR = latest_lstm_run(RESULTS_DIR, lookback=LOOKBACK, horizon=HORIZON, eps_mode=MODE, epsilon=EPS, strict=False)

if RUN_DIR is None or not RUN_DIR.exists():
    raise SystemExit("Kein *_lstm Run gefunden. Bitte Block 3/4/6 vorher einmal ausführen.")

print("Aktives RUN_DIR:", RUN_DIR)

Aktives RUN_DIR: C:\Users\jacin\DL_PROJECT\finance_transformer_lstm\LSTM\results\2026-01-03_21-02-35_lstm


In [5]:
# === ARTEFAKTE EINLESEN ===
# Wir laden 'evaluation.json', das in den vorherigen Blocks Stück für Stück gefüllt wurde.
ev_path = RUN_DIR / "evaluation.json"
if not ev_path.exists():
    raise SystemExit(f"evaluation.json fehlt in {RUN_DIR}. Wurden Block 4 und 6 ausgeführt?")

ev    = jread(ev_path)
cfg   = ev.get("config", {})
metrics = (ev.get("metrics", {}) or {}).get("test", {})
thr_sel = ev.get("threshold_selection", {})
calib   = ev.get("calibration", {})
backtest_gross = ev.get("backtest", {})

# Fallback: Label-Parameter, falls oben nicht gefunden
if HORIZON is None:
    HORIZON = int(((ev.get("label_resolved_from") or {}).get("horizon")) or cfg.get("horizon"))
if MODE is None:
    MODE = (ev.get("label_resolved_from") or {}).get("mode") or cfg.get("epsilon_mode")
if EPS is None:
    EPS = float((ev.get("label_resolved_from") or {}).get("epsilon") or cfg.get("epsilon"))

In [6]:
# === KOSTEN-DATEN LADEN ===
# Aus Block 6: Sensitivitäts-Ergebnisse
sens_path = RUN_DIR / "cost_sensitivity.csv"
sens_df = pd.read_csv(sens_path) if sens_path.exists() else None

MAIN_RT = 15.0  # Unser Standard-Szenario für den Bericht: 15 bps Roundtrip
MAIN_SLIP_PER_LEG = 2.0

cost_pick = {}
if sens_df is not None and not sens_df.empty:
    # Wir filtern auf das realistische Modell "Entry@t+1"
    df_t1_exact = sens_df[sens_df["model"] == "Entry@t+1"]
    df_t1_prefix = sens_df[sens_df["model"].astype(str).str.startswith("Entry@t+1")]
    df_t1 = df_t1_exact if not df_t1_exact.empty else df_t1_prefix

    if df_t1.empty:
        df_t1 = sens_df.copy() # Falls Filter fehlschlägt, nimm alles

    # Wir suchen die Zeile, die den 15bps am nächsten kommt
    df_t1["rt_diff"] = (df_t1["roundtrip_bps"] - MAIN_RT).abs()
    row = df_t1.sort_values(["rt_diff", "roundtrip_bps"]).iloc[0].to_dict()

    # Struktur für den Report
    cost_pick = dict(
        model=row["model"], 
        roundtrip_bps=float(row["roundtrip_bps"]),
        trades=int(row.get("trades", 0)), 
        exposure=float(row.get("exposure", np.nan)),
        turnover=float(row.get("turnover", np.nan)),
        CAGR=float(row["CAGR"]), 
        Sharpe=float(row["Sharpe"]), 
        MaxDD=float(row["MaxDD"]),
        final_equity=float(row["final_equity"]),
    )

In [7]:
# === BASELINES BERECHNEN ===
# Wir berechnen einfache Benchmarks, um zu sehen, ob das komplexe LSTM überhaupt Mehrwert liefert.
# WICHTIG: Das muss auf exakt denselben Daten (Test-Split) passieren!

TRAIN_CSV = Path(cfg.get("train_csv", ""))
features_list = cfg.get("features", None)

if not features_list:
    # Falls keine Feature-Liste im Config, aus YAML holen
    yml = ROOT / f"data/features_{FEATURESET}.yml"
    if yml.exists():
        meta = yaml.safe_load(open(yml, "r")) or {}
        features_list = meta.get("features", [])

# Originaldaten laden und splitten
df_all = pd.read_csv(TRAIN_CSV, index_col=0, parse_dates=True).sort_index()
X_all = df_all[features_list].copy()
y_all = df_all["target"].astype(int).copy()

# Zeitlicher Split (muss identisch zu Notebook 3 sein: 70/15/15)
n = len(df_all)
n_train = int(n * 0.70)
n_val   = int(n * 0.15)
n_test  = n - n_train - n_val

X_train, y_train = X_all.iloc[:n_train],              y_all.iloc[:n_train]
X_val,   y_val   = X_all.iloc[n_train:n_train+n_val], y_all.iloc[n_train:n_train+n_val]
X_test,  y_test  = X_all.iloc[n_train+n_val:],        y_all.iloc[n_train+n_val:]

# Preprocessing und Scaling
LB = int(cfg.get("lookback", LOOKBACK))
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score

# Scaler nur auf Train fitten!
scaler = StandardScaler().fit(X_train)
Xtr_s = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
Xva_s = pd.DataFrame(scaler.transform(X_val),   index=X_val.index,   columns=X_val.columns)
Xte_s = pd.DataFrame(scaler.transform(X_test),  index=X_test.index,  columns=X_test.columns)

# Da LSTMs die ersten LB-1 Zeilen verlieren, passen wir den Vergleich hier auch an
# (Obwohl LogReg theoretisch alles nutzen könnte, für fairen Vergleich cutten wir)
tail = slice(LB-1, None)
ytr_tail, yva_tail, yte_tail = y_train.iloc[tail], y_val.iloc[tail], y_test.iloc[tail]
Xtr_tail, Xva_tail, Xte_tail = Xtr_s.iloc[tail],   Xva_s.iloc[tail],   Xte_s.iloc[tail]

# 1. Baseline: Always-Up / Buy-And-Hold (Rate der positiven Labels)
pos_rate_test = float(yte_tail.mean())
auprc_always_up = pos_rate_test

# 2. Baseline: Logistische Regression (Lineares Referenz-Modell)
logit = LogisticRegression(max_iter=200)
logit.fit(Xtr_tail, ytr_tail)
proba_lr = logit.predict_proba(Xte_tail)[:,1]
auprc_lr = float(average_precision_score(yte_tail, proba_lr))

# 3. Baseline: Simpler MACD Indikator (falls vorhanden)
# Wir testen, ob ein einfacher Indikator besser ist als das komplexe Modell
if "macd_diff" in df_all.columns:
    macd_diff = df_all.loc[Xte_tail.index, "macd_diff"].astype(float).fillna(0.0)
    auprc_macd = float(average_precision_score(yte_tail, macd_diff.values))
else:
    auprc_macd = 0.0

baselines_tbl = pd.DataFrame([
    {"baseline": "Always-Up (Prior)", "auprc": auprc_always_up, "pos_rate": pos_rate_test,
     "lift_factor": 1.0},
    {"baseline": "Logistic Regression", "auprc": auprc_lr, "pos_rate": pos_rate_test,
     "lift_factor": (auprc_lr / max(pos_rate_test, 1e-12))},
    {"baseline": "Simple MACD", "auprc": auprc_macd, "pos_rate": pos_rate_test,
     "lift_factor": (auprc_macd / max(pos_rate_test, 1e-12))}
])

In [8]:
# === REPORT DATEN ZUSAMMENFÜHREN ===
# Wir bauen ein großes Dictionary 'kpis', in dem alle wichtigen Infos gesammelt sind.

kpis = {
    "run_dir": str(RUN_DIR),
    "generated_utc": datetime.utcnow().isoformat(timespec="seconds") + "Z",
    "data": {
        "ticker": cfg.get("ticker"),
        "interval": cfg.get("interval"),
        "period": [cfg.get("start"), cfg.get("end")],
        "horizon": cfg.get("horizon"),
        "lookback": cfg.get("lookback"),
        "featureset": cfg.get("featureset"),
        "features_used": ev.get("features_used"),
    },
    "label": ev.get("label_resolved_from"),
    "calibration": {
        "chosen": calib.get("chosen"),
        "val_brier": calib.get("val_brier"),
        "test_brier": calib.get("test_brier"),
        "note": "Kalibrierung auf Validation Set, Test Set Werte nur zur Info."
    },
    "threshold": {
        "strategy": thr_sel.get("strategy"),
        "threshold": thr_sel.get("threshold"),
        "val_mcc": thr_sel.get("val_mcc"),
        "test_pred_pos_rate": thr_sel.get("test_pred_pos_rate"),
    },
    "classification_test": {
        "roc_auc": metrics.get("roc_auc"),
        "auprc": metrics.get("auprc"),
        "brier": metrics.get("brier"),
        "balanced_accuracy": metrics.get("balanced_accuracy"),
        "mcc": metrics.get("mcc"),
    },
    "backtest_gross": backtest_gross,     # Backtest Ergebniss ohne Kosten
    "backtest_cost_pick": cost_pick,      # Ergebnisse mit Kosten (Realistisch)
    "baselines": baselines_tbl.to_dict(orient="records")
}

In [9]:
# === REPORT GENERIEREN (MARKDOWN) ===
# Wir schreiben die gesammelten Daten in eine sauber formatierte Markdown-Datei.

fig_dir = RUN_DIR / "figures"
# Dateipfade zu den Bildern, die wir einbinden wollen
figs = {
    "roc": fig_dir / "roc_test.png",
    "pr":  fig_dir / "pr_test.png",
    "equity_cost":  fig_dir / "equity_costed.png",
}

def _rel(p: Path) -> str:
    # Hilfsfunktion für relative Pfade (damit der Report portabel ist)
    return str(p.relative_to(RUN_DIR)) if p.exists() else str(p)

report_md = RUN_DIR / "REPORT_block7.md"
lines = []

# Header
lines.append(f"# Block 7 – Abschluss-Report\n")
lines.append(f"- **Generiert:** {kpis['generated_utc']}")
lines.append(f"- **Run:** `{RUN_DIR.name}`")
lines.append(f"- **Modell-Setup:** Ticker `{kpis['data']['ticker']}` | Horizon `{kpis['data']['horizon']}` | Lookback `{kpis['data']['lookback']}`")
lines.append("---")

# Klassifikations-Metrics
lines.append("## 1. Klassifikations-Leistung (Test Set)")
m = kpis["classification_test"]
lines.append(f"Wir bewerten, wie gut das Modell Wahrscheinlichkeiten vorhersagt.")
lines.append(f"- **AUROC**: `{m['roc_auc']:.3f}` (Fläche unter ROC Kurve, 0.5 = Zufall)")
lines.append(f"- **AUPRC**: `{m['auprc']:.3f}` (Fläche unter Precision-Recall Kurve, Basisrate ≈ {kpis['baselines'][0]['pos_rate']:.2f})")
lines.append(f"- **MCC**: `{m['mcc']:.3f}` (Matthews Correlation Coefficient)")

lines.append(f"\n### Grafiken\n")
lines.append(f"![ROC]({_rel(figs['roc'])})  \n![PR]({_rel(figs['pr'])})\n")

# Backtests
lines.append("## 2. Finanzielle Performance (Backtest)")
cp = kpis["backtest_cost_pick"]
if cp:
    lines.append(f"Simuliertes Handelsergebnis mit **{cp['roundtrip_bps']}** bps Roundtrip-Kosten und T+1 Entry:")
    lines.append(f"- **CAGR**: `{cp['CAGR']:.2%}` (Jährliche Rendite)")
    lines.append(f"- **Sharpe Ratio**: `{cp['Sharpe']:.2f}` (Risikoadjustierte Rendite)")
    lines.append(f"- **Max Drawdown**: `{cp['MaxDD']:.2%}` (Maximaler zwischenzeitlicher Verlust)")
    lines.append(f"- **Endkapital**: `{cp['final_equity']:.2f}` (Start = 1.00)")
    lines.append(f"\n![Equity Trace]({_rel(figs['equity_cost'])})\n")
else:
    lines.append("*(Keine Kostendaten gefunden)*\n")

# Baselines
lines.append("## 3. Vergleich mit Benchmarks")
lines.append("Ist das Modell besser als simple Methoden? (Lift Factor > 1.0)")
lines.append("| Baseline | AUPRC | Lift Factor |")
lines.append("|---|---|---|")
for b in kpis["baselines"]:
    lines.append(f"| {b['baseline']} | {b['auprc']:.3f} | {b['lift_factor']:.2f}x |")

lines.append("\n---")
lines.append("**Ende des Berichts.**")

# Datei schreiben
report_md.write_text("\n".join(lines), encoding="utf-8")
print("✓ REPORT geschrieben:", report_md)

# JSON Dump für spätere Analyse
(RUN_DIR / "REPORT_block7_kpis.json").write_text(json.dumps(kpis, indent=2), encoding="utf-8")

# CSV Zeile für Sammel-Auswertungen
try:
    kpi_rows = {
        "roc_auc": metrics.get("roc_auc"), 
        "auprc": metrics.get("auprc"), 
        "mcc": metrics.get("mcc"),
        "cost_CAGR": cp.get("CAGR") if cp else None,
        "cost_Sharpe": cp.get("Sharpe") if cp else None,
    }
    pd.DataFrame([kpi_rows]).to_csv(RUN_DIR / "kpis_block7.csv", index=False)
except Exception as e:
    print("Fehler beim CSV-Export:", e)

✓ REPORT geschrieben: C:\Users\jacin\DL_PROJECT\finance_transformer_lstm\LSTM\results\2026-01-03_21-02-35_lstm\REPORT_block7.md
